In [26]:
import tensorflow as tf
import numpy as np
import tfgraphviz as tfg
from tensorboard import notebook
from datetime import datetime

In [27]:
# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)
    
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch
        
def logdirname(root_logdir='./'):
    now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
    logdir = "{}/run-{}/".format(root_logdir, now)
    return logdir

In [28]:
reset_graph()

n_inputs = 28*28 # MNIST
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [29]:
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="weights")
        b = tf.Variable(tf.zeros([n_neurons]), name="biases")
        z = tf.matmul(X, W) + b
        if activation=="relu":
            return tf.nn.relu(z)
        else:
            return z

In [30]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [31]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.keras.layers.Dense(n_hidden1, name="hidden1", activation=tf.nn.relu)(X)
    hidden2 = tf.keras.layers.Dense(n_hidden2, name="hidden2", activation=tf.nn.relu)(hidden1)
    logits = tf.keras.layers.Dense(n_outputs, name="outputs")(hidden2)

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

loss_summary = tf.summary.scalar('loss', loss)
file_writer = tf.summary.FileWriter(logdirname(), tf.get_default_graph())

In [32]:
n_epochs = 400
batch_size = 50

with tf.Session() as sess:
    init.run()
    idx = 0
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            if(idx%10==0):
                summary_str = loss_summary.eval(feed_dict={X: X_batch, y: y_batch})
                file_writer.add_summary(summary_str, idx)                
            idx += 1
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)
    save_path = saver.save(sess, "./model_with_dense_layer_final1.ckpt")

0 Train accuracy: 0.86 Test accuracy: 0.9026
1 Train accuracy: 0.94 Test accuracy: 0.923
2 Train accuracy: 0.94 Test accuracy: 0.9328
3 Train accuracy: 0.92 Test accuracy: 0.9382


In [33]:
file_writer.close()

In [34]:
with tf.Session() as sess:
    saver.restore(sess, "./model_with_dense_layer_final1.ckpt") # or better, use save_path
    X_new_scaled = X_test[:20]
    Z = logits.eval(feed_dict={X: X_new_scaled})
    y_pred = np.argmax(Z, axis=1)

INFO:tensorflow:Restoring parameters from ./model_with_dense_layer_final1.ckpt


In [35]:
print("Predicted classes:", y_pred)
print("Actual classes:   ", y_test[:20])

Predicted classes: [7 2 1 0 4 1 4 9 6 9 0 6 9 0 1 5 9 7 3 4]
Actual classes:    [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]


In [36]:
g = tfg.board(tf.get_default_graph())
g.view()

'G.gv.pdf'

In [37]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
#%reload_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [38]:
%tensorboard --logdir ./

Reusing TensorBoard on port 6006 (pid 14016), started 1:08:02 ago. (Use '!kill 14016' to kill it.)